<a href="https://colab.research.google.com/github/OJB-Quantum/Notebooks-for-Ideas/blob/main/H_Bridge_Inverter_GDS_Using_OpenLane_PDK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ===========================================================
# Colab Notebook — H-Bridge Inverter (Sky130 · OpenLane 2)
# ===========================================================
# Author : Onri Jay Benally
# Date   : 2025-06-02
# -----------------------------------------------------------
# What this notebook does
# 1) Installs OpenLane 2 + Sky130 PDK via Volare
# 2) Creates a structural-Verilog four-transistor H-bridge
# 3) Generates a full-chip, DRC-clean GDS
# 4) Renders a quick-look PNG of the layout
# 5) Provides a download link for the GDS
# -----------------------------------------------------------
# ↑ Runtime ▸ Change runtime type ▸ (GPU works, CPU is fine)
# ===========================================================

In [2]:
%%bash
# 1️⃣  Linux packages
sudo apt-get update -qq
sudo apt-get install -y git build-essential make klayout imagemagick


Reading package lists...
Building dependency tree...
Reading state information...
build-essential is already the newest version (12.9ubuntu3).
make is already the newest version (4.3-4.1build1).
klayout is already the newest version (0.26.2-3build3).
git is already the newest version (1:2.34.1-1ubuntu1.12).
imagemagick is already the newest version (8:6.9.11.60+dfsg-1.3ubuntu0.22.04.5).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [3]:
# 2️⃣  Python packages  (IPython cell, NOT inside %%bash)
%pip install -q "openlane==2.*" "volare>=0.20.4,<0.21" gdspy


In [4]:
%%bash
# 3️⃣  Set and create the PDK folder
export PDK_ROOT=/content/pdks
mkdir -p "$PDK_ROOT"


In [5]:
%%bash
# 4️⃣  Enable the Sky130 kit (OpenLane-tested commit)
export PDK_ROOT=/content/pdks
volare enable --pdk-root "$PDK_ROOT" --pdk sky130 \
    cd1748bb197f9b7af62a54507de6624e30363943


Version cd1748bb197f9b7af62a54507de6624e30363943 enabled for the sky130 PDK.


In [6]:
# 5️⃣  Tell Python / OpenLane where the PDK is
import os, pathlib, sys
os.environ["PDK_ROOT"] = "/content/pdks"
os.environ["PDK"]      = "sky130A"
assert (pathlib.Path(os.environ["PDK_ROOT"]) / "sky130A").exists(), "PDK missing!"
print("✅  PDK ready.")


✅  PDK ready.


In [7]:
from pathlib import Path            # path handling
import textwrap                     # handy for dedenting multi-line strings
import json                         # read/write JSON


In [8]:
from pathlib import Path
import textwrap, json               # ← corrected import line

proj = Path("/content/h_bridge_project")
src  = proj / "src"
src.mkdir(parents=True, exist_ok=True)

(src / "h_bridge.v").write_text(textwrap.dedent(r'''
module h_bridge (
    input  wire in_a, input wire in_b,
    output wire out_p, output wire out_n,
    supply1 vccd1, supply0 vssd1
);
    sky130_fd_pr__pfet_01v8 p1 (.G(in_a), .D(out_p), .S(vccd1), .B(vccd1));
    sky130_fd_pr__pfet_01v8 p2 (.G(in_b), .D(out_n), .S(vccd1), .B(vccd1));
    sky130_fd_pr__nfet_01v8 n1 (.G(in_b), .D(out_p), .S(vssd1), .B(vssd1));
    sky130_fd_pr__nfet_01v8 n2 (.G(in_a), .D(out_n), .S(vssd1), .B(vssd1));
endmodule
'''))

(src / "pin_order.cfg").write_text("vccd1 vssd1 in_a in_b out_p out_n\n")

(proj / "config.json").write_text(json.dumps({
    "DESIGN_NAME":   "h_bridge",
    "PDK":           "sky130A",
    "PDK_ROOT":      "/content/pdks",
    "VERILOG_FILES": "src/h_bridge.v",
    "FP_PIN_ORDER_CFG": "pin_order.cfg",
    "FP_CORE_UTIL":  20,
    "RUN_MAGIC_DRC": "on",
    "RUN_LVS":       "on",
    "RUN_CVC":       "on",
    "RUN_KLAYOUT":   "on"
}, indent=2))

print("🎯  Project folder ready:", proj)


🎯  Project folder ready: /content/h_bridge_project


In [9]:
import subprocess, textwrap, os, pathlib, pprint, sys

bash_script = textwrap.dedent("""
    set -x                           # echo every command
    export PDK_ROOT=/content/pdks
    export PDK=sky130A
    openlane --design-dir /content/h_bridge_project --run-tag colab
""")

print("🔍  Re-running OpenLane with full stdout/stderr capture …\n")
result = subprocess.run(
    ["bash", "-c", bash_script],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

print(result.stdout)           # show everything OpenLane printed
print("\n⛔  Exit code:", result.returncode)


🔍  Re-running OpenLane with full stdout/stderr capture …

+ export PDK_ROOT=/content/pdks
+ PDK_ROOT=/content/pdks
+ export PDK=sky130A
+ PDK=sky130A
+ openlane --design-dir /content/h_bridge_project --run-tag colab
[04:07:34] ERROR    No config file(s) have been provided.         __main__.py:77


⛔  Exit code: 1


In [10]:
# 8️⃣  Quick PNG preview + download
import glob, gdspy, matplotlib.pyplot as plt
gds = glob.glob("/content/h_bridge_project/runs/*/gds/*.gds")[0]
lib = gdspy.GdsLibrary(); lib.read_gds(gds)
cell = lib.top_level()[0]
plt.figure(figsize=(6,6), dpi=200)
for polys in cell.get_polygons(by_spec=True).values():
    for p in polys: plt.fill(p[:,0], p[:,1], alpha=.8, lw=0)
plt.axis('equal'); plt.axis('off'); plt.title("Sky130 H-Bridge — Layout")
plt.show()
from google.colab import files; files.download(gds)


IndexError: list index out of range